# Music Festival Analysis

The festival organizers want to improve the festival. They wanted to understand which performance are bringing them the most profits and where they are falling short.
Using the class_music_festival database, I will perform exploratory analysis using SQL to help the organizers get a sense of the festival's performance and answer some of the onganizer's burning questions.

In [1]:
%LOAD postgresql host=localhost dbname=class_music_festival -- loading database

#### The organizers had announced a giveaway for the first 10 customers. They wanted to know the attendee's names to announce the winners.

In [19]:
SELECT people.name AS customer_name
FROM people
WHERE people.id IN (SELECT purchases.person_id
                    FROM purchases
                    ORDER BY purchases.date ASC
                    LIMIT 10);

customer_name
Osderbiq
Logsalhet
Rykcer
Nercux Toc
Mehex
Beq
Fa
Diaspol
Duffixav Balgab
Fel Dow


#### When was the first ticket to a performance at the venue BMI purchased?

In [17]:
SELECT purchases.date
FROM tickets
  JOIN purchases
    ON purchases.id = tickets.purchase_id
  JOIN performances
    ON performances.id = tickets.performance_id
  JOIN venues
    ON venues.id = performances.venue_id
WHERE venues.name = 'BMI'
ORDER BY purchases.date ASC
LIMIT 1;

date
2011-01-01 07:13:51


#### When is the first show (band name, venue name and start time) that 'Osderbiq ' has a ticket to?

In [21]:
SELECT bands.name AS band_name, venues.name AS venue_name, performances.start
FROM tickets
  JOIN purchases
    ON purchases.id = tickets.purchase_id
  JOIN people
    ON people.id = purchases.person_id
  JOIN performances
    ON performances.id = tickets.performance_id
  JOIN venues
    ON venues.id = performances.venue_id
  JOIN bands
    ON bands.id = performances.band_id
WHERE people.name = 'Osderbiq '
ORDER BY performances.start ASC
LIMIT 1;

band_name,venue_name,start
An Horse,Austin Ventures,2011-09-16 12:20:00


#### What is the name of the person who spent the most (and how much did they spend)?

In [24]:
SELECT people.id, people.name, SUM(price) as total_spent
FROM people
  JOIN purchases
    ON purchases.person_id = people.id
  JOIN tickets
    ON tickets.purchase_id = purchases.id
GROUP BY people.id
ORDER BY total_spent DESC
LIMIT 1;

id,name,total_spent
426,Vebil Miesys,765


#### What is the name of the band performing at the AMD venue that had the highest fee?

In [22]:
SELECT bands.name
FROM performances
  JOIN venues 
    ON venues.id = performances.venue_id
  JOIN bands
    ON bands.id = performances.band_id
WHERE venues.name = 'AMD'
ORDER BY fee DESC
LIMIT 1;

name
Ryan Bingham & The Dead Horses


#### How many tickets were sold for the performance that started at 2011-09-18 20:30:00?

In [23]:
SELECT COUNT(*)
FROM tickets
  JOIN performances
    ON performances.id = tickets.performance_id
WHERE performances.start = '2011-09-18 20:30:00';

count
825


#### Which bands performed at the venue with more than 2000 capacity?

In [15]:
SELECT DISTINCT bands.name AS band_name
FROM bands
WHERE bands.id IN (SELECT performances.band_id
                    FROM performances
                    WHERE venue_id = (SELECT venues.id
                                        FROM venues
                                        WHERE venues.capacity > 2000));

band_name
Brady Rymer
Heidi Swedberg
Mariana Iranzi
Peter DiStefano & Tor
Quinn Sullivan
Recess Monkey
Sara Hickman
The Q Brothers
The School of Rock


#### Which performance and venue had the highest revenue?

In [36]:
SELECT performances.id, venues.name AS venue_name, SUM(price) as revenue
FROM performances
  JOIN tickets
    ON tickets.performance_id = performances.id
  JOIN venues
    ON venues.id = performances.venue_id
GROUP BY performances.id, venues.name
ORDER BY revenue DESC
LIMIT 1;

id,venue_name,revenue
41,Vista Equity,57968


#### Which performance (and by which band) was the most profitable?

In [28]:
SELECT performances.id, bands.name AS band_name, (SUM(tickets.price)-SUM(DISTINCT bands.fee)) AS profit
FROM performances
  JOIN tickets
    ON tickets.performance_id = performances.id
  JOIN bands
    ON bands.id = performances.band_id
GROUP BY performances.id, bands.name
ORDER BY profit DESC
LIMIT 1;

id,band_name,profit
41,Preservation Hall Jazz Band and The Del McCoury Band,53305


#### Which band was the least profitable for the festival?

In [30]:
SELECT bands.id, bands.name AS band_name, (SUM(tickets.price)-SUM(DISTINCT bands.fee)) AS profit
FROM bands
    JOIN performances
      ON bands.id = performances.band_id
    JOIN tickets
      ON performances.id = tickets.performance_id
GROUP BY bands.id, bands.name
ORDER BY profit
LIMIT 1;

id,band_name,profit
87,WAX,2279


#### Which venues were oversold and what were their capacities?

In [31]:
SELECT performances.id, venues.name, SUM(DISTINCT capacity) AS capacity, (COUNT(DISTINCT ticketnum) - SUM(DISTINCT capacity)) AS overcapacity
FROM venues
  JOIN performances
    ON venues.id = performances.venue_id
  JOIN tickets
    ON performances.id = tickets.performance_id
GROUP BY performances.id, venues.name
HAVING (COUNT(DISTINCT ticketnum) - SUM(DISTINCT capacity)) > 0;

id,name,capacity,overcapacity
1,AMD,700,164
2,AMD,700,67
3,AMD,700,120
4,AMD,700,114
5,AMD,700,116
45,AMD,700,181
46,AMD,700,110
69,AMD,700,142
81,AMD,700,105
88,AMD,700,85


#### What was the total revenue from ticket sales each month?

In [32]:
SELECT EXTRACT(MONTH FROM purchases.date) AS month, SUM(tickets.price) AS revenue
FROM tickets
  JOIN purchases
    ON purchases.id = tickets.purchase_id
GROUP BY month
ORDER BY month;

month,revenue
1,264510
2,226199
3,247469
4,254352
5,281448
6,258480
7,260668
8,263922
9,127317


#### What was the average purchase total each month?

In [34]:
SELECT EXTRACT(MONTH FROM purchases.date) AS month, ROUND((SUM(price::decimal)/COUNT(DISTINCT purchases.id)),2) AS avg_purchase
FROM tickets
  JOIN purchases
    ON purchases.id = tickets.purchase_id
GROUP BY month;

month,avg_purchase
1,201.76
2,203.23
3,204.69
4,202.35
5,207.25
6,203.21
7,196.43
8,205.23
9,200.5


## Conclusion

Looking at the analysis above, the festival organizers will need to ensure that they are not overselling tickets for the AMD venue since overcrowding may result in decrease of attendee's enjoyment of the performance. FRom the analysis we have concluded that Preservation Hall Jazz Band and The Del McCoury Band and Vista Equity had the highest revenue, while the band WAX is the least profitable. The organizers can consider tracking attendee's enjoyment of each performance, in a way that makes it possible to ask which bands/venues are the most popular.